<a href="https://colab.research.google.com/github/jjc16/Geometry_Tools/blob/main/Hypercube_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hypercube Utilities

This notebook contains a series of functions that build up to the final function: **find_all_hypercubes**. This function allows users to input a list of points and return a count of all hypercubes (of various side lengths) inscribed on the points. 

This notebook was inspired by problem that I saw on Youtube (can't find the link) to an interview question for one of the FAANG companies where the interviewee is asked to find the number of squares in grid of points. This question seemed boring and relatively simple (the only hard part was that it had to work after 45 minutes of effort), so I made the problem harder and put it into the notebook below. 

**Extensions:** the function will currently only return hypercubes of the same dimension as the input list of points. A good extension of this would be to have the function return all hypercubes of dimension $[2, ... n]$ where n is the dimension of the list. This would take some work (because you would essentially have to search through $(n \space choose \space k)$ subspaces (with $n$ the dimension of the main space and k the dimension of the subspace) for each $k$. But, it would be doable with some effort and for loops :-)

Also, the function should have some error checking on the list of points and more test cases.



## Helper functions

See code on github for more details.

In [5]:
list2 = [[1,1,1], [1,1,2], [1,2,1], [1,2,2], [2,1,1], [2,1,2], [2,2,1], [2,2,2], [3,1,1], [3,1,2], [3,2,1], [3,2,2]]


In [2]:
def min_all_dims(list_):
  #Finds the minimum distance between points in all lists along all
  #coordinate axis
  #
  #Example: [[1,1],[2,3]] is 1
  # [[1,1], [4,5]] is 3
  if not list_:
    return None

  out = []
  for ii in range(len(list_[0])):
    tmp = [pt[ii] for pt in list_]
    t2 = max(tmp)
    t1 = min(tmp)
    out.append(t2 - t1)
  return min(out)


In [10]:
min_all_dims(list2)
min_all_dims([[1,1,1], [5,4,3]])

2

In [ ]:
def find_coord(point1, point2):
  #Find the coordinate axis along which to search for "lines"
  for ii in range(len(point1)):
    if point1[ii] != point2[ii]:
      return ii
  return []

In [ ]:
def gen_tst_pts(start, end, coord):
    #Figure out which points between each coordinate are actually in a line
    #and which might be missing
    tst_pts=[]
    for ii in range(start[coord], end[coord]+1):
      tmp = start.copy()
      tmp[coord] = ii
      tst_pts.append(tmp)

    return tst_pts

In [ ]:
def test_pts(list_, tst_pts):
  return [pt in list_ for pt in tst_pts]

In [ ]:
def line_in_list(list_, start, end, coord):
  tst_pts = gen_tst_pts(start, end, coord)
  out = test_pts(list_, tst_pts)

  return all(out)

In [ ]:
def hyper_cube_corners(start, length):
  if len(start) == 1:
    tmp = start.copy()
    out = []
    out.append(tmp)
    tmp2 = list(map(lambda x:x+length, tmp))
    out.append(tmp2) 
    return out
  else:
    tmp = hyper_cube_corners(start[:-1], length)
    out = []
    for item in tmp:
      item1 = item.copy()
      item1.append(start[-1])
      item2 = item.copy()
      tmp = start[-1] + length
      item2.append(tmp)
      out.append(item1)
      out.append(item2)
    return out
      

In [ ]:
def find_lines(corners):
  lines = []
  for ii in range(len(corners)):
    for jj in range(ii, len(corners)):
      tmp = [int(not(corners[ii][kk] == corners[jj][kk])) for kk in range(len(corners[ii]))]
      # print(tmp)
      if sum(tmp) == 1:
        lines.append(tuple((corners[ii], corners[jj])))

  return lines

In [ ]:
def find_all_cube_one_size(list_, sz):
  list_.sort()
  count = 0
  for l in list_:
      corners = hyper_cube_corners(l, sz)
      lines = find_lines(corners)
      ln_lst = []
      for ln in lines:
        coord = find_coord(ln[0], ln[1])
        pts = gen_tst_pts(ln[0], ln[1], coord)
        tmp = test_pts(list_, pts)
        tmp = all(tmp)
        ln_lst.append(tmp)
      if all(ln_lst):
        count += 1

  return count


## Main function

In [ ]:
def find_all_hypercubes(list_):
  count = 0
  out_dict = {}
  sz = min_all_dims(list_)
  for ii in range(1,sz+1):
    tmp = find_all_cube_one_size(list_, ii)
    out_dict[ii] = tmp
    count += tmp

  return count, out_dict


## Test Cases

In [ ]:
list_ = [[1,1], [1,2], [1,3], [2,1], [2,2], [2,3], [3,1], [3,2], [3,3], [4,1], [4,2], [4,3], [5,1], [5,2], [5,3]]
find_all_cube_one_size(list_, 1)
find_all_hypercubes(list_)

(11, {1: 8, 2: 3})

In [4]:
list2 = [[1,1,1], [1,1,2], [1,2,1], [1,2,2], [2,1,1], [2,1,2], [2,2,1], [2,2,2], [3,1,1], [3,1,2], [3,2,1], [3,2,2]]
# find_all_hypercubes(list2)


## Extensions

This code can be extended to also find the cubic manifolds of lower dimension in the space. For example, in 5-dimensional space, we have cubic manifolds that are 2-d, 3-d, and 4-d.

I will probably try to complete this project when I have sufficient time. One hint for doing this is the following partial algorithm:

1. Generate the indices of a unit n-dimensional hypercube (where n is the maximal dimension in the space). 

2. Replace the zeros of each grid point that has at least two nonzero values and at least one zero value with an iteration through the span of values in the dimensions that have nonzero entries.

3. Iterate through those values. **Hint**: use the hypercube indices to generate a lower dimensional projection of the hypercube (with the zero values representing the kernel) The hypercubes found will be $k$-dimensional cubic manifolds in the $n$-dimensional lattice hyperspace (where $k<n$).

## Experimental Code

In [29]:
def generate_subspaces_list(vertices):
  return [ver for ver in vertices if 0 in ver and sum(ver)>=2]


In [27]:
ind = hyper_cube_corners([0,0,0,0], 1)
print(ind)
print(generate_subspaces_list(ind))
out = generate_subspaces_list(ind)

[[0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 1], [0, 1, 0, 0], [0, 1, 0, 1], [0, 1, 1, 0], [0, 1, 1, 1], [1, 0, 0, 0], [1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 1], [1, 1, 0, 0], [1, 1, 0, 1], [1, 1, 1, 0], [1, 1, 1, 1]]
[[0, 0, 1, 1], [0, 1, 0, 1], [0, 1, 1, 0], [0, 1, 1, 1], [1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 1], [1, 1, 0, 0], [1, 1, 0, 1], [1, 1, 1, 0]]


In [10]:
def all_possible_combos_2(a, b):
  pass 
  

In [38]:
def all_possible_combos(lst):
  if len(lst)==0:
    return []
  elif len(lst)==1:
    return(list)
  elif len(lst)==2:
    # print(lst)
    out = [[ii,jj] for ii in lst[0] for jj in lst[1]]
    # print(out)
    return out
  else:
    tmp = all_possible_combos(lst[:-1])
    # print(tmp)
    # print(lst[-1])
    out = [t+[l] for t in tmp for l in lst[-1]]
    return out

In [44]:
a=[1,2,3]
b=[4,5,6] 
c=[7,8,9]
d = [1,1,2,1]
tst = all_possible_combos([a,b,c,d])

In [7]:
 xdef generate_subspace(list_, idx):
  out = [[] for _ in range(len(list_))]
  for ii in range(len(idx)):
    if idx[ii]==0:
      for jj in range(len(list_)):
        out[jj].append(list_[jj][ii])

  return out




In [9]:
list4 = [[1,2,3,4], [1,6,7,8], [1,10,11,12]]
generate_subspace(list4, [0,0,1,1])

[[1, 2], [1, 6], [1, 10]]

In [32]:
out = [[] for _ in range(4)]

In [33]:
out

[[], [], [], []]